In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

In [ ]:
from context import Context
from utils.utils_chain import WrapperAddress as Address, Account, hex_to_string
from utils.utils_chain import nominated_amount

context = Context()

In [3]:
from contracts.farm_contract import FarmContract
from utils.contract_retrievers import retrieve_farm_by_address
# farm_contracts = context.get_contracts(config.FARMS_V2)

In [ ]:
farm_contract: FarmContract = context.deploy_structure.get_deployed_contract_by_index(config.FARMS_V2, 2)
print(f"Using {farm_contract.address} : {farm_contract.farmToken}")

In [ ]:
farm_contract: FarmContract = retrieve_farm_by_address("")

set new rewards per block

In [ ]:
farm_contract.set_rewards_per_block(context.deployer_account, context.network_provider.proxy, nominated_amount(14000))

start contracts

In [ ]:
farm_contract.resume(context.deployer_account, context.network_provider.proxy)

produce rewards

In [ ]:
farm_contract.start_produce_rewards(context.deployer_account, context.network_provider.proxy)

upgrade contract

In [ ]:
pair_address = farm_contract.get_lp_address(context.network_provider.proxy)
tx_hash = farm_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, 
                                         config.FARM_V2_BYTECODE_PATH, [pair_address, context.deployer_account.address.bech32()])

In [ ]:
from contracts.farm_contract import FarmContractVersion
context.deployer_account.sync_nonce(context.network_provider.proxy)
# upgrade all farms
farms : list[FarmContract] = context.get_contracts(config.FARMS_V2)
for farm in farms:
    farm.version = FarmContractVersion.V2Boosted
    pair = farm.get_lp_address(context.network_provider.proxy)
    tx_hash = farm.contract_upgrade(context.deployer_account, context.network_provider.proxy, 
                                         config.FARM_V3_BYTECODE_PATH, [], no_init=True)
    context.network_provider.check_complex_tx_status(tx_hash, "farm upgrade")

In [ ]:
# resume all farms
context.deployer_account.sync_nonce(context.network_provider.proxy)
contracts : list[FarmContract] = context.get_contracts(config.FARMS_V2)
for contract in contracts:
    tx_hash = contract.resume(context.deployer_account, context.network_provider.proxy)

change owner

In [ ]:
tx_hash = farm_contract.update_owner_or_admin(context.deployer_account, context.network_provider.proxy, "erd19uysnnzwnzm3klq8qzv7zprw2d3yj38hx0ac9cuqvl0j2jr5kp2sflvmyv")

resume contract

In [ ]:
farm_contract.resume(context.deployer_account, context.network_provider.proxy)

Collect user accounts

In [ ]:
from utils.utils_scenarios import collect_farm_contract_users
from multiversx_sdk import ApiNetworkProvider

mainnet_api = ApiNetworkProvider("https://api.multiversx.com")
fetched_users = collect_farm_contract_users(100, farm_contract.address, farm_contract.farmingToken, farm_contract.farmToken,
                                            mainnet_api, context.network_provider.proxy)

Used users

In [ ]:
from utils.decoding_structures import FARM_TOKEN_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes, base64_to_hex, get_all_token_nonces_details_for_account
from utils.utils_scenarios import FetchedUser

index = 0
users: List[FetchedUser] = fetched_users.get_users_with_farm_tokens()
if not users:
    raise Exception('No users found for the given criteria')

user = users[index].address
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user.bech32(), context.network_provider.proxy)
print(f'Account: {user.bech32()}')
print(f'Farming Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user.bech32(), context.network_provider.proxy)
print(f'Farm Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
    print(f'\t\t{decode_merged_attributes(base64_to_hex(token["attributes"]), FARM_TOKEN_ATTRIBUTES)}')

Find user by token

In [ ]:
from multiversx_sdk.network_providers import ApiNetworkProvider
from utils.utils_chain import dec_to_padded_hex, WrapperAddress, get_all_token_nonces_details_for_account, decode_merged_attributes, base64_to_hex
from utils.decoding_structures import FARM_TOKEN_ATTRIBUTES

migration_nonce = 152776

mainnet_api = ApiNetworkProvider("https://api.multiversx.com")
for nonce in reversed(range(migration_nonce)):
    url = f'nfts/{farm_contract.farmToken}-{dec_to_padded_hex(nonce)}/accounts'
    try:
        response = mainnet_api.do_get_generic(url)
        user_found = ""
        for entry in response:
            if WrapperAddress(entry['address']).is_smart_contract():
                continue
            user_found = entry['address']
        
        if user_found:
            tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_found, context.network_provider.proxy)
            for token in tokens_in_account:
                token_decoded_attributes = decode_merged_attributes(base64_to_hex(token["attributes"]), FARM_TOKEN_ATTRIBUTES)
                print(f'Farm Tokens nonce {dec_to_padded_hex(nonce)}: {token_decoded_attributes}')
    except Exception as e:
        print(f'Error fetching {url}: {e}')
        continue

In [ ]:
from utils.decoding_structures import FARM_TOKEN_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes, base64_to_hex

user = Account(pem_file=config.DEFAULT_ACCOUNTS)
user.address = WrapperAddress("erd15u0ulntwpjl32m7afnnatz48kp8sec6euj64vl97vxhynxcmt9jq0n350m")
user.sync_nonce(context.network_provider.proxy)

tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user.address.bech32(), context.network_provider.proxy)
print(f'Account: {user.address.bech32()}')
print(f'Farming Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user.address.bech32(), context.network_provider.proxy)
print(f'Farm Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
    print(f'\t\t{decode_merged_attributes(base64_to_hex(token["attributes"]), FARM_TOKEN_ATTRIBUTES)}')

Transfer position

In [ ]:
from utils.utils_tx import multi_esdt_transfer, ESDTToken
from utils.utils_chain import Account
fetched_user = users[0]
user_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
user_account.address = fetched_user.address
user_account.sync_nonce(context.network_provider.proxy)

full_token_id = fetched_user.farm_tokens[0].get('tokenIdentifier')
token_id = '-'.join(full_token_id.split('-')[:-1])
token = ESDTToken(token_id, fetched_user.farm_tokens[0].get('nonce'), int(fetched_user.farm_tokens[0].get('balance'))//2)
print(f'Sending: {token.get_token_data()} from {fetched_user.address.bech32()}')
multi_esdt_transfer(context.network_provider.proxy, 1000000, user_account, users[1].address, [token])

Enter farm

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import EnterFarmEvent

# user = fetched_users[randint(0, len(fetched_users.get_users_with_farming_tokens()) - 1)].address
# user = fetched_users.get_users_with_both_tokens()[0].address
user_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
user_account.address = user
user_account.sync_nonce(context.network_provider.proxy)

farming_tk_balance = 0
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user_account.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} farming tokens in account')
for token in tokens_in_account:
    if int(token['balance']) > 0:
        farming_tk_balance = int(token['balance'])
        break

if not farming_tk_balance:
    raise Exception("Not enough farming token balance")

farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_account.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} farm tokens in account')
for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        break

# use only 10% of the balance
event = EnterFarmEvent(farm_contract.farmingToken, 0, farming_tk_balance // 10,
                       farm_contract.farmToken, farm_tk_nonce, farm_tk_balance)

initial_enter: bool = False if farm_tk_nonce else True
tx_hash = farm_contract.enterFarm(context.network_provider, user_account, event, initial=initial_enter)

In [ ]:
proxy_contract.set_transfer_role_wrapped_lp_token(context.deployer_account, context.network_provider.proxy, proxy_contract.address)

Claim rewards

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

# user = fetched_users[randint(0, len(fetched_users.get_users_with_farming_tokens()) - 1)].address
# user = fetched_users.get_users_with_both_tokens()[0].address
user_account.sync_nonce(context.network_provider.proxy)

farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_account.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        break

if not farm_tk_nonce:
    raise Exception("Not enough farm token balance")

event = ClaimRewardsFarmEvent(farm_tk_balance, farm_tk_nonce, '')

tx_hash = farm_contract.claimRewards(context.network_provider, user_account, event)

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

user_account = Account(pem_file="~/Documents/sh1.pem")
user_account.address = user.address
user_account.sync_nonce(context.network_provider.proxy)

event = ClaimRewardsFarmEvent(0, 0, '', user_account.address.bech32())

tx_hash = farm_contract.claim_boosted_rewards(context.network_provider, user_account, event)

Get user farm position

In [ ]:
# user_address = fetched_users.get_users_with_both_tokens()[0].address.bech32()
user_address = user.bech32()
print(user_address)
farm_contract.get_user_total_farm_position(user_address, context.network_provider.proxy)

Exit rewards

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ExitFarmEvent

# user = fetched_users[randint(0, len(fetched_users.get_users_with_farming_tokens()) - 1)].address
# user = fetched_users.get_users_with_both_tokens()[0].address
user_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
user_account.address = user
user_account.sync_nonce(context.network_provider.proxy)

farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_account.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        break

if not farm_tk_nonce:
    raise Exception("Not enough farm token balance")

event = ExitFarmEvent(farm_contract.farmToken, farm_tk_balance, farm_tk_nonce, '')

tx_hash = farm_contract.exitFarm(context.network_provider, user_account, event)

Get user farm position

In [ ]:
# user_address = fetched_users.get_users_with_both_tokens()[0].address.bech32()
user_address = user_account.address.bech32()
print(user_address)
farm_contract.get_user_total_farm_position(user_address, context.network_provider.proxy)

SHADOWFORK CONTROL

In [ ]:
from contracts.builtin_contracts import SFControlContract

epocs_to_advance = 7

sf_control_contract = SFControlContract(config.SF_CONTROL_ADDRESS)
current_epoch = context.network_provider.proxy.get_network_status(1).epoch_number
print(f'Current epoch: {current_epoch}')
context.deployer_account.sync_nonce(context.network_provider.proxy)
sf_control_contract.epochs_fast_forward(context.deployer_account, context.network_provider.proxy, epocs_to_advance, 9)

wait for epoch fast forward

In [ ]:
from time import sleep

target_epoch = current_epoch + epocs_to_advance
while current_epoch < target_epoch:
    current_epoch = context.network_provider.proxy.get_network_status(1).epoch_number
    print(f'Current epoch: {current_epoch}')
    sleep(10)
print('Reached target epoch!')

Trackers

In [ ]:
from trackers.farm_economics_tracking import FarmEconomics
from contracts.farm_contract import FarmContractVersion

farm_tracker = FarmEconomics(farm_contract.address, FarmContractVersion.V2Boosted, context.network_provider)

In [ ]:
farm_tracker.update_tracking_data()
farm_tracker.report_current_tracking_data()

In [ ]:
farm_tracker.update_tracking_data()
farm_tracker.report_current_tracking_data()

State

In [ ]:
current_week = farm_contract.get_current_week(context.network_provider.proxy)
print(current_week)

In [ ]:
energy_for_user_in_week = farm_contract.get_user_energy_for_week(user.bech32(), proxy=context.network_provider.proxy, week=current_week)
print(energy_for_user_in_week)

In [ ]:
user_progress = farm_contract.get_current_claim_progress_for_user(user.bech32(), context.network_provider.proxy)
print(user_progress)

In [ ]:
last_week = farm_contract.get_last_active_week_for_user(user.bech32(), context.network_provider.proxy)
print(last_week)

In [ ]:
from tools.runners.farm_runner import get_farm_addresses_from_chain
from contracts.farm_contract import FarmContract

farm_addresses = get_farm_addresses_from_chain("v2")

for farm_address in farm_addresses:
    farm_contract = FarmContract.load_contract_by_address(farm_address)
    stats = farm_contract.get_all_stats(context.network_provider.proxy)

    print(f'Farm: {farm_address}')
    print(f'Farm identity: {farm_contract.get_config_dict()}')
    print(f'Stats: {stats}')
    print('')


New Farm setup

In [ ]:
FARM_TOKEN_NAME = ""
FARM_TOKEN_TICKER = ""
POOL_ADDRESS = ""
LOCK_EPOCHS = 1440
BOOSTED_YIELD_REWARDS = 6000
BOOSTED_YIELD_FACTORS = [2, 1, 0, 1, 1]
REWARDS_PER_BLOCK = 1
PENALTY_PERCENT = 300
MINIMUM_FARMING_EPOCH = 7

In [ ]:
from contracts.farm_contract import FarmContract, FarmContractVersion
from contracts.pair_contract import PairContract
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract

pair_contract = PairContract.load_contract_by_address(POOL_ADDRESS)
print(f'Pair contract: {pair_contract.address} {pair_contract.lpToken}')
energy_factory: SimpleLockEnergyContract
energy_factory = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

farm_contract: FarmContract
farm_contract = FarmContract(pair_contract.lpToken, "", energy_factory.base_token, "", FarmContractVersion.V2Boosted)

In [ ]:
hash, address = farm_contract.contract_deploy(context.deployer_account, context.network_provider.proxy, 
                                              config.FARM_V3_BYTECODE_PATH,
                                              [POOL_ADDRESS, context.admin_account.address.bech32()])
farm_contract.address = address

In [ ]:
farm_contract.register_farm_token(context.deployer_account, context.network_provider.proxy, [FARM_TOKEN_NAME, FARM_TOKEN_TICKER])

In [ ]:
pair_contract.whitelist_contract(context.deployer_account, context.network_provider.proxy, farm_contract.address)

In [ ]:
farm_contract.set_energy_factory_address(context.deployer_account, context.network_provider.proxy, energy_factory.address)

In [ ]:
farm_contract.set_locking_address(context.deployer_account, context.network_provider.proxy, energy_factory.address)

In [ ]:
energy_factory.add_sc_to_whitelist(context.deployer_account, context.network_provider.proxy, farm_contract.address)

In [ ]:
farm_contract.set_lock_epochs(context.deployer_account, context.network_provider.proxy, LOCK_EPOCHS)

In [ ]:
farm_contract.set_boosted_yields_rewards_percentage(context.deployer_account, context.network_provider.proxy, BOOSTED_YIELD_REWARDS)

In [ ]:
farm_contract.set_boosted_yields_factors(context.deployer_account, context.network_provider.proxy, BOOSTED_YIELD_FACTORS)

In [ ]:
farm_contract.set_rewards_per_block(context.deployer_account, context.network_provider.proxy, 1)

In [ ]:
farm_contract.set_penalty_percent(context.deployer_account, context.network_provider.proxy, 300)

In [ ]:
farm_contract.set_minimum_farming_epochs(context.deployer_account, context.network_provider.proxy, 7)

In [ ]:
energy_factory.set_transfer_role_locked_token(context.deployer_account, context.network_provider.proxy, [farm_contract.address])